In [ ]:
import pandas as pd
import numpy as np
import folium
import matplotlib.pyplot as plt
from folium.plugins import HeatMap
from sklearn.preprocessing import MinMaxScaler


def enhance_probability(probs, new_min=0.1, new_max=0.95, power=1.5):
    probs = np.array(probs)
    scaler = MinMaxScaler((0, 1))
    scaled = scaler.fit_transform(probs.reshape(-1, 1)).flatten()
    powered = np.power(scaled, power)
    final_scaler = MinMaxScaler((new_min, new_max))
    enhanced_probs = final_scaler.fit_transform(powered.reshape(-1, 1)).flatten()
    return enhanced_probs


def generate_heatmap_from_csv(csv_path: str, enhance=True):
    df = pd.read_csv(csv_path)

    columns_to_keep = ['latitude', 'longitude', 'probability']

    dropped_cols = [col for col in df.columns if col not in columns_to_keep]
    if dropped_cols:
        print(f"🧹 Удалены ненужные колонки: {dropped_cols}")

    df = df[columns_to_keep].copy()

    df.drop_duplicates(subset=['latitude', 'longitude'], inplace=True)


    print(f"📄 Загружено {len(df)} точек из {csv_path}")
    print("🔍 df.columns:", list(df.columns))

    df = df[['latitude', 'longitude', 'probability']].copy()
    df.columns = ['latitude', 'longitude', 'probability'] 

    df = df[
        (df['latitude'].between(-90, 90)) &
        (df['longitude'].between(-180, 180))
    ]
    if df.empty:
        raise ValueError("❌ Нет допустимых координат после фильтрации")

    df['probability_enhanced'] = (
        enhance_probability(df['probability'].values) if enhance else df['probability'].values
    )

    map_center = [df['latitude'].median(), df['longitude'].median()]
    m = folium.Map(location=map_center, zoom_start=10, tiles='cartodbpositron')

    heat_data = df[['latitude', 'longitude', 'probability_enhanced']].values.tolist()
    HeatMap(
        heat_data,
        radius=15,
        blur=20,
        min_opacity=0.4,
        gradient={
            '0.0': '#00ff00',   
            '0.2': '#ccff00',   
            '0.4': '#ffff00',  
            '0.6': '#ff9900',  
            '0.8': '#ff0000',   
        }
    ).add_to(m)



    legend_html = '''
    <div style="position: fixed; bottom: 20px; left: 20px; width: 240px;
        background-color: white; border: 2px solid #ddd; border-radius: 8px;
        padding: 12px; z-index:9999; font-family: Arial, sans-serif;">
        <div style="text-align: center; font-weight: bold; font-size: 16px; margin-bottom: 10px;">
            Landslide Risk Levels
        </div>
        <div><span style="background-color:#00ff00;width:20px;height:20px;display:inline-block;"></span>
            &nbsp;🟢 Very Low (0.10 - 0.20)</div>
        <div><span style="background-color:#ccff00;width:20px;height:20px;display:inline-block;"></span>
            &nbsp;🟡 Low (0.20 - 0.40)</div>
        <div><span style="background-color:#ffff00;width:20px;height:20px;display:inline-block;"></span>
            &nbsp;🟠 Moderate (0.40 - 0.60)</div>
        <div><span style="background-color:#ff9900;width:20px;height:20px;display:inline-block;"></span>
            &nbsp;🔴 High (0.60 - 0.80)</div>
        <div><span style="background-color:#ff0000;width:20px;height:20px;display:inline-block;"></span>
            &nbsp;🚨 Very High (0.80 - 0.95)</div>
    </div>
    '''

    m.get_root().html.add_child(folium.Element(legend_html))

    output_html = "heatmap_landslide.html"
    m.save(output_html)
    print(f"✅ Тепловая карта сохранена в {output_html}")

    return m


if __name__ == "__main__":
    generate_heatmap_from_csv("predicted_landslide_risk.csv", enhance=True)


🧹 Удалены ненужные колонки: ['cat', 'value', 'slope', 'aspect', 'flowacc', 'twi', 'spi', 'profc', 'planc', 'ndvi', 'rainfall', 'soilmoisture', 'dist2river', 'latitude.1', 'longitude.1', 'cluster', 'label', 'risk']
📄 Загружено 270062 точек из predicted_landslide_risk.csv
🔍 df.columns: ['latitude', 'longitude', 'probability']
✅ Тепловая карта сохранена в heatmap_landslide.html
